In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:18pt;}
div.output {font-size:19pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv (지역명, 평당분양가격, 연도, 월)
- 지역명2 : 지역명필드는 라벨인코딩하여 추가
- 평당분양가격n : 평당분양가격의 normalization 스케일 조정하여 추가
- 평당분양가격s : 평당분양가격의 standardization 스케일 조정하여 추가
- 연도n : 연도의 normalization 스케일 조정하여 추가
- 연도s : 연도의 standardization 스케일 조정하여 추가
- 월n   : 월의 normalization 스케일 조정하여 추가
- 월s   : 월의 standardization 스케일 조정하여 추가

## 1. 데이터 가져오기

In [7]:
import pandas as pd
csvdata = pd.read_csv(r'D:\ai\Downloads\shareData\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv',
                encoding='cp949')
df = csvdata.melt(id_vars="지역", var_name="연도월", value_name="평당분양가격")
df["연도"] = df["연도월"].apply(lambda x : int(x.split("년")[0]))
df["월"] = df["연도월"].apply(lambda x : int(x.split("년")[-1][:-1]))
df.drop('연도월', axis=1, inplace=True)
df = df.rename(columns={'지역':'지역명'})
df.head(2)

,지역명,평당분양가격,연도,월
0,서울,18189,2013,12
1,부산,8111,2013,12


## 2. 지역명의 라벨 인코딩

In [8]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le = LabelEncoder()
df['지역명2'] = le.fit_transform(df['지역명'])
df.head()

,지역명,평당분양가격,연도,월,지역명2
0,서울,18189,2013,12,8
1,부산,8111,2013,12,7
2,대구,8080,2013,12,5
3,인천,10204,2013,12,11
4,광주,6098,2013,12,4


# 3. normalization 스케일 조정

In [104]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

year_scaler = MinMaxScaler()
# year_scaler.fit(np.array(df['연도']).reshape(-1,1))
# df['연도n']=year_scaler.transform(np.array(df['연도']).reshape(-1,1))
year_scaler.fit_transform(np.array(df['연도']).reshape(-1,1))

mon_scaler = MinMaxScaler()
mon_scaler.fit( np.array(df['월']).reshape(-1,1) )
df['월n']=pd.Series(mon_scaler.transform(np.array(df['월']).reshape(-1,1)).reshape(-1))

price_scaler = MinMaxScaler()
price_scaler.fit( np.array(df['평당분양가격']).reshape(-1,1) )
df['평당분양가격n']=price_scaler.transform(np.array(df['평당분양가격']).reshape(-1,1))

df.loc[::100]

,지역명,지역명2,평당분양가격,평당분양가격n,평당분양가격s,연도,연도n,연도s,월,월n,월s
0,서울,8,18189,0.835270,3.297226,2013,0.0,-2.366432,12,1.000000,1.767475
100,경남,2,6610,0.066463,-0.621248,2014,0.5,-0.591608,5,0.363636,-0.294579
200,전남,12,5768,0.010557,-0.906191,2014,0.5,-0.591608,11,0.909091,1.472896
300,충남,15,6975,0.090698,-0.497727,2015,1.0,1.183216,5,0.363636,-0.294579


In [15]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# 열별 스케일 조정
year_scaler = MinMaxScaler()
df['연도n'] = year_scaler.fit_transform(df[['연도']])

mon_scaler = MinMaxScaler()
df['월n'] = mon_scaler.fit_transform( df[['월']])

price_scaler = MinMaxScaler()
df['평당분양가격n'] = price_scaler.fit_transform( df[['평당분양가격']] )

df.loc[::100]
# 2025년 3월에 경남(2번지역) (2, year_scaler.transform(2025), mon_scaler.transform(3))

,지역명,평당분양가격,연도,월,지역명2,연도n,월n,평당분양가격n
0,서울,18189,2013,12,8,0.0,1.000000,0.835270
100,경남,6610,2014,5,2,0.5,0.363636,0.066463
200,전남,5768,2014,11,12,0.5,0.909091,0.010557
300,충남,6975,2015,5,15,1.0,0.363636,0.090698


In [17]:
# 변수들 전체 스케일 조정
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler = MinMaxScaler()
df[['연도n','월n','평당분양가격n']] = scaler.fit_transform(df[['연도','월','평당분양가격']])
df.head()

,지역명,평당분양가격,연도,월,지역명2,연도n,월n,평당분양가격n
0,서울,18189,2013,12,8,0.0,1.0,0.835270
1,부산,8111,2013,12,7,0.0,1.0,0.166124
2,대구,8080,2013,12,5,0.0,1.0,0.164066
3,인천,10204,2013,12,11,0.0,1.0,0.305093
4,광주,6098,2013,12,4,0.0,1.0,0.032468


# standardization 

In [19]:
from sklearn.preprocessing import StandardScaler
import numpy as np
# 열별 스케일 조정
year_scaler = StandardScaler()
year_scaler.fit(np.array(df['연도']).reshape(-1,1))
df['연도s']= year_scaler.transform(np.array(df['연도']).reshape(-1,1))

mon_scaler = StandardScaler()
mon_scaler.fit( np.array(df['월']).reshape(-1,1) )
df['월s']=mon_scaler.transform(np.array(df['월']).reshape(-1,1))

price_scaler = StandardScaler()
price_scaler.fit( np.array(df['평당분양가격']).reshape(-1,1) )
df['평당분양가격s']= price_scaler.transform(np.array(df['평당분양가격']).reshape(-1,1) )

df = df[['지역명','지역명2',
    '평당분양가격','평당분양가격n','평당분양가격s',
    '연도','연도n','연도s','월','월n','월s']]
df.loc[::100]

,지역명,지역명2,평당분양가격,평당분양가격n,평당분양가격s,연도,연도n,연도s,월,월n,월s
0,서울,8,18189,0.835270,3.297226,2013,0.0,-2.366432,12,1.000000,1.767475
100,경남,2,6610,0.066463,-0.621248,2014,0.5,-0.591608,5,0.363636,-0.294579
200,전남,12,5768,0.010557,-0.906191,2014,0.5,-0.591608,11,0.909091,1.472896
300,충남,15,6975,0.090698,-0.497727,2015,1.0,1.183216,5,0.363636,-0.294579


In [20]:
# 변수들 전체 스케일 조정
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = MinMaxScaler()
df[['연도s','월s','평당분양가격s']] = scaler.fit_transform(df[['연도','월','평당분양가격']])
df.head()

,지역명,지역명2,평당분양가격,평당분양가격n,평당분양가격s,연도,연도n,연도s,월,월n,월s
0,서울,8,18189,0.835270,0.835270,2013,0.0,0.0,12,1.0,1.0
1,부산,7,8111,0.166124,0.166124,2013,0.0,0.0,12,1.0,1.0
2,대구,5,8080,0.164066,0.164066,2013,0.0,0.0,12,1.0,1.0
3,인천,11,10204,0.305093,0.305093,2013,0.0,0.0,12,1.0,1.0
4,광주,4,6098,0.032468,0.032468,2013,0.0,0.0,12,1.0,1.0
